In [1]:
import import_ipynb
from loadData import raw_df, ad_info_columns, user_info_columns, context_info_columns, shop_info_columns, result_columns,ad_info_df, user_info_df, context_info_df, shop_info_df, result_df
from Util import getAllTypesofCategory, getAllTypesOfProperty

importing Jupyter notebook from loadData.ipynb
importing Jupyter notebook from Util.ipynb


In [2]:
from sklearn.preprocessing import Imputer, MinMaxScaler, OneHotEncoder, LabelEncoder
import numpy as np
import itertools
import pandas as pd
import collections
from scipy.sparse import csr_matrix, save_npz
import time

## impute missing value with most_frequent strategy

In [3]:
contain_missing_value_columns = ["item_brand_id","item_city_id","item_sales_level","user_gender_id","user_age_level",
                                "user_occupation_id","user_star_level","shop_review_positive_rate",
                                "shop_score_service","shop_score_delivery","shop_score_description"]
other_missing_value_columns = ["predict_category_property"]

In [4]:
data_imputer = Imputer(missing_values=-1 , strategy='most_frequent', axis=0)
raw_df[contain_missing_value_columns] = data_imputer.fit_transform(raw_df[contain_missing_value_columns])

## processing the object column

In [ ]:
object_columns = ["item_category_list", "item_property_list", "predict_category_property"]

In [6]:
# split category_list to multi features
def extract_item_category_list_func(item, num):
    items = item.split(';')
    if num >=2:
        if len(items) > 2:
            return items[num]
        else:
            return str(0)
    else:
        return items[num]
def generate_item_category_features():
    raw_df['item_one_level_category'] = raw_df['item_category_list'].apply(lambda item: extract_item_category_list_func(item, 0))
    raw_df['item_second_level_category'] = raw_df['item_category_list'].apply(lambda item: extract_item_category_list_func(item, 1))
    raw_df['item_third_level_category'] = raw_df['item_category_list'].apply(lambda item: extract_item_category_list_func(item, 2))
#     raw_df.drop('item_category_list', axis=1)
generate_item_category_features()

In [7]:
need_onehot_columns = ['item_brand_id','item_city_id','user_gender_id','item_one_level_category',
                       'item_second_level_category']
special_onehot_columns = ['item_property_list_array', 'item_third_level_category']
other_columns = ['context_timestamp','predict_category_property','is_trade']
need_scale_columns = ['item_price_level','item_sales_level','item_collected_level','item_pv_level',
                     'user_age_level','user_occupation_id','user_star_level','context_page_id',
                     'shop_review_num_level','shop_review_positive_rate','shop_star_level','shop_score_service',
                    'shop_score_description']

## One hot common column

In [8]:
raw_df = raw_df.drop(['item_category_list','item_property_list'], axis=1).reset_index()

for i in ['item_brand_id','item_city_id','user_gender_id']:
    raw_df[i] = raw_df[i].apply(lambda item : str(item))

_result_df = pd.get_dummies(raw_df[need_onehot_columns], prefix=need_onehot_columns)

save_npz('../processed_data/need_onehot_columns.npz', csr_matrix(_result_df.values))

### third level item process

In [9]:
# third_level_items = raw_df['item_third_level_category'].unique()

# lb = LabelEncoder()
# _tmp = lb.fit_transform(third_level_items)
# result = lb.transform(raw_df['item_third_level_category'].values)

# ohenc = OneHotEncoder()
# ohenc.fit(_tmp.reshape(-1,1))
# result = ohenc.transform(result.reshape(-1,1))

# tmp_df = pd.DataFrame(data=result.toarray(), columns=['item_third_level_None', 'item_third_level_8868887661186419229', 'item_third_level_6233669177166538628'])

# tmp_df.to_csv('../processed_data/item_third_level_one_hot.csv', sep=' ',float_format='%d',index=False)

##  scale data in need_scale_column

In [10]:
# # min-max normalization
# _tmpdf = raw_df[need_scale_columns]
# # normalized_df = (_tmpdf - _tmpdf.min())/(_tmpdf.max() - _tmpdf.min())

# # mean normalization
# normalized_df = (_tmpdf - _tmpdf.mean())/_tmpdf.std()
# normalized_df.to_csv('../processed_data/scaled.csv', sep=' ',index=False)

##  process date column